# Get Data From Workspaces

In this notebook we, log on to our PowerBI workspaces, and iterate over them and fecth all data on all avalailiable `datasets` and `reports`

First we install the needed libraries, but only once

In [ ]:
#%pip install azure-identity

Use the InteractiveBrowserCredential to get the access token:

In [ ]:
from azure.identity import InteractiveBrowserCredential
import requests

# Initialize the credential
credential = InteractiveBrowserCredential()

# Define the scope for Power BI API
scope = ["https://analysis.windows.net/powerbi/api/.default"]

# Get the access token
token = credential.get_token(*scope).token

print(f"Token: {token}")

Now with the obtained token, let's see the list of workspaces

In [ ]:
# Use the token to make API requests
headers = {
    'Authorization': f'Bearer {token}'
}

# Example: Get workspaces
url = 'https://api.powerbi.com/v1.0/myorg/groups'
response = requests.get(url, headers=headers)
workspaces = response.json()

print(f"Found  {len(workspaces['value'])} availiable workpaces")

for workspace in workspaces['value']:
    print(f"Workspace Name: {workspace['name']}, Workspace ID: {workspace['id']}")

Now, let's define some functions we can use to retrieve the data that we want

In [ ]:

# Function to get all workspaces
def get_workspaces():
    url = 'https://api.powerbi.com/v1.0/myorg/groups'
    response = requests.get(url, headers=headers)
    return response.json()

# Function to get reports in a workspace
def get_reports(workspace_id):
    url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports'
    response = requests.get(url, headers=headers)
    return response.json()

# Function to get datasets in a workspace
def get_datasets(workspace_id):
    url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets'
    response = requests.get(url, headers=headers)
    return response.json()

# Function to get the last refresh time of a dataset
def get_dataset_refresh_history(workspace_id, dataset_id):
    url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/refreshes'
    params = {
        '$top': "1"
    }
    response = requests.get(url, params=params, headers=headers)
    return response.json()


# Fetch all workspaces
workspaces = get_workspaces()

# List to store report information
report_list = []

# List to store dataset information
datasets_list = []

# Iterate through workspaces and fetch reports and datasets
for workspace in workspaces['value']:
    workspace_id = workspace['id']
    workspace_type = workspace["type"]
    workspace_name = workspace['name']

    print(f"Workspace Name: {workspace_name}\tWorkspace Type: {workspace_type}\tWorkspace Id: {workspace_id}")
   
    # Fetch reports
    reports = get_reports(workspace_id)
    for report in reports['value']:
        report_name = report['name']
        report_id = report['id']
        report_type = report["reportType"]
        # Not all attributes are avaliable on all reports, so here we use get to be more fault tolerant
        report_web_url = report.get("webUrl")
        report_dataset_workspace_id = report.get("datasetWorkspaceId")
        report_dataset_id = report.get("datasetId")

        # print(f"Report Name: {report['name']}, Report ID: {report['id']}, Workspace ID: {workspace_id}, Owner: {report['createdBy']}, Last Refresh: {report['lastModifiedDateTime']}")
        # Add report information to the list
        report_list.append({
            "workspace_name": workspace_name,
            "workspace_id": workspace_id,
            "name": report_name,
            "id": report_id,
            "report_type": report_type,
            "web_url": report_web_url,
            "dataset_workspace_id": report_dataset_workspace_id,
            "dataset_id": report_dataset_id
        })


    # Fetch datasets (semantic models)
    datasets = get_datasets(workspace_id)
    for dataset in datasets['value']:
        dataset_name = dataset['name']
        dataset_id = dataset['id']
        #refresh_history = get_dataset_refresh_history(workspace_id, dataset_id)
        #last_refresh = refresh_history['value'][0]['endTime'] if refresh_history['value'] else 'No refresh history'
        # print(f"Dataset Name: {dataset['name']}, Dataset ID: {dataset['id']}, Workspace ID: {workspace_id}, Last Refresh: {last_refresh}")
        print(f"Dataset Name: {dataset['name']}, Dataset ID: {dataset['id']}, Workspace ID: {workspace_id}")
         # Add dataset information to the list
        datasets_list.append({
            "workspace_name": workspace_name,
            "workspace_id": workspace_id,
            "name": dataset_name,
            "id": dataset_id
        })

Let's save the data, for easier access in `Excel`

In [ ]:
# Use Pandas DataFrame for easy handling
import pandas as pd
import os

df_reports = pd.DataFrame(report_list)
df_datasets = pd.DataFrame(datasets_list)

# Check if the directory exists, if not, create it
directory = './data'
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the data as Excel files
df_reports.to_excel(f'{directory}/reports.xlsx', index=False)
df_datasets.to_excel(f'{directory}/datasets.xlsx', index=False)

print(f'Written output to {directory}')